#### Código de Hamming (7, 4) ####
Para codificar a fonte $s$ para ser transmitida pelo código de Hamming como $t$, aplica-se a seguinte transformação linear:
$t = G^ts$

onde G é a matriz Geradora. Ela pode ser expressa da seguinte maneira:
$G^t = \begin{bmatrix}
I_4\\ 
P
\end{bmatrix}$

onde $I_4$ é a identidade e P a matriz de paridade (combinação dos bits três a três).

$ P = \begin{bmatrix}
1 & 1 & 1 & 0\\ 
0 & 1 & 1 & 1\\ 
1 & 0 & 1 & 1
\end{bmatrix}$

Para decodificar o código recebido, é preciso verificar as somas dos bits três a três. Se tudo estiver correto, não há anomalias e a resposta é 
$z = \begin{bmatrix}
0\\ 
0\\ 
0
\end{bmatrix}$

Logo, para decodificar, aplica-se a transformação $H = \begin{bmatrix}
P & I_3
\end{bmatrix}$ ao código recebido $t$.

$z = Ht$

Isso funciona porque $HG^t = 0$. Logo, se não houver anomalias na transmissão, o resultado será $z = \begin{bmatrix}
0\\ 
0\\ 
0
\end{bmatrix}$. Se somente um bit for modificado, é possível descobri-lo pois cada anomalia indica unicamente o bit a ser corrigido para que a soma das paridades esteja correta. No entanto, se dois ou mais bits estiverem trocados, a simples troca indicada pela análise da anomalia trocará o bit incorreto, e ficaremos com um erro a mais.

In [150]:
import numpy as np
import re

#### Verificação do código de Hamming (Q1.4)####

In [70]:
P = np.array([[1,1,1,0], [0,1,1,1], [1,0,1,1]], dtype='b')
G_t = np.concatenate([np.eye(4), P])
H = np.concatenate([P, np.eye(3)], axis=1)
print(H)
z = (np.dot(H, G_t)% 2).astype('b')
print(z)

[[ 1.  1.  1.  0.  1.  0.  0.]
 [ 0.  1.  1.  1.  0.  1.  0.]
 [ 1.  0.  1.  1.  0.  0.  1.]]
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


#### Decodificação dos códigos recebidos (Q1.5)####

In [196]:
def decode(r):
    r = np.asarray([list(r)], dtype='b')
    P = np.array([[1,1,1,0], [0,1,1,1], [1,0,1,1]])
    H = np.concatenate([P, np.eye(3)], axis=1)
    
    z = np.dot(H, r.T) % 2
    return z

def flip_code(z, r):
    H = np.concatenate([P, np.eye(3)], axis=1).T
    idx = np.where(np.all(H == z.T, axis=1))
    if not idx[0].any():
        return r[:4]
    idx = idx[0][0]
    flipped = int(r[idx],2) ^ 1
    r = r[:idx] + str(flipped) + r[idx+1:]
    double_check = decode(r)
    if not double_check.any():
        print("Codigo consertado")
    else:
        print("oh oh")
    return r[:4]

codes = ['1101011', '0110110', '0100111', '1111111']
for r in codes:
    z = decode(r)
    print("Anomalia %s" % z.T)
    code = flip_code(z, r)
    print("O código %s decodifica como %s\n" % (r, code))

Anomalia [[ 0.  1.  1.]]
Codigo consertado
O código 1101011 decodifica como 1100

Anomalia [[ 1.  1.  1.]]
Codigo consertado
O código 0110110 decodifica como 0100

Anomalia [[ 0.  0.  1.]]
Codigo consertado
O código 0100111 decodifica como 0100

Anomalia [[ 0.  0.  0.]]
O código 1111111 decodifica como 1111



#### Cálculo do erro (Q1.6 a Q1.8)####
Um erro de um bloco (7,4) acontece se dois ou mais bits vierem trocados. Se somente um bit estiver trocado, este é consertado pela confirmação da paridade. A probabilidade de erro de um bloco (7,4) utilizando código de Hamming é de:

$p_B = \binom{7}{2}f^2 + \binom{7}{3}f^3 + \binom{7}{4}f^4 + \binom{7}{5}f^5 + \binom{7}{6}f^6 + \binom{7}{7}f^7$

O termo de *leading order* é $\binom{7}{2}f^2 = 21 f^2$ 

Portanto, o Hamming funciona da seguinte maneira:
1. Se errar somente 1 bit, o decoder consertará.
2. Se errar 2 bits, o decoder tenta consertar o bit errado e 3 bits acabam estando errados no final. Logo, considerando somente o termo de *leading order*, a chance de 1 bit estar errado ao final é de $\frac{3}{7}$ da chance do bloco estar errado. Ou seja:

$p_b = \frac{3}{7} 21 f^2 = \frac{63}{7} f^2 = 9 f^2$

In [231]:
# Iterate over all possible binary numbers to check

noise = []
for i in range(0,2**7):
    r = np.binary_repr(i, width=7)
    z = decode(r)
    if not z.any():
        noise.append(r)
print("Existem %d noise vectors: %s" % (len(noise), noise))

Existem 16 noise vectors: ['0000000', '0001011', '0010111', '0011100', '0100110', '0101101', '0110001', '0111010', '1000101', '1001110', '1010010', '1011001', '1100011', '1101000', '1110100', '1111111']
